In [38]:
%load_ext autoreload
%autoreload 2
import sys
import gc

from multiprocessing import cpu_count, Pool
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils

from tqdm import tqdm

sys.path.append('../')
from wiki.utils import clock
from wiki import rnn, rnn_predict, newphet, val, submissions
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
torch.manual_seed(1)
torch.cuda.set_device(0)

In [25]:
batch_size = 512
pred_len = 60

In [26]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_1.csv').fillna(0)

In [27]:
X = train_df.drop('Page', axis=1).values
X, scaler = rnn.scale_values(X)

Use all the data to train this time.

In [7]:
trainloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-pred_len,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

Its crucial that the `testloader` is not shuffled, as we'll use the page names directly from the train csv to index them later.

In [8]:
testloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:,:]).float(),
        torch.zeros(X.shape[0])
    ),
    batch_size=batch_size, shuffle=False
)

In [9]:
loss_func = nn.L1Loss()
model = rnn.RNN(loss_func=loss_func).cuda()

Epochs chosen by amount of epochs to best val loss on CV data.

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
save_best_path = base_dir+'rnn_v2_lr1.mdl'
with clock():
    model.fit(trainloader, None, optimizer=optimizer, num_epochs=18, save_best_path=save_best_path)
save_best_path = base_dir+'rnn_v2_predictor_lr1_weights.mdl'
torch.save(model.state_dict(), save_best_path)


EPOCH 1
Running average loss: 0.380938
VALIDATION LOSS: 0.456770
Elapsed time 161.36574244499207 seconds

EPOCH 2
Running average loss: 0.387845
VALIDATION LOSS: 0.499010
Elapsed time 161.3883695602417 seconds

EPOCH 3
Running average loss: 0.372888
VALIDATION LOSS: 0.459744
Elapsed time 162.1795949935913 seconds

EPOCH 4
Running average loss: 0.363239
VALIDATION LOSS: 0.449002
Elapsed time 162.0177822113037 seconds

EPOCH 5
Running average loss: 0.353778
VALIDATION LOSS: 0.443414
Elapsed time 162.78928780555725 seconds

EPOCH 7
Running average loss: 0.352572
VALIDATION LOSS: 0.443005
Elapsed time 162.35685443878174 seconds

EPOCH 8
Running average loss: 0.352043
VALIDATION LOSS: 0.436838
Elapsed time 162.50207376480103 seconds

EPOCH 9
Running average loss: 0.349686
VALIDATION LOSS: 0.437846
Elapsed time 162.65406918525696 seconds

EPOCH 10
Running average loss: 0.342699
VALIDATION LOSS: 0.440097
Elapsed time 163.68144512176514 seconds

EPOCH 11
Running average loss: 0.340099
VALIDAT

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=8, save_best_path=save_best_path)
save_best_path = base_dir+'rnn_v2_predictor_lr2_weights.mdl'
torch.save(model.state_dict(), save_best_path)


EPOCH 1
Running average loss: 0.318066
VALIDATION LOSS: 0.419108
Elapsed time 163.71369981765747 seconds

EPOCH 2
Running average loss: 0.315877
VALIDATION LOSS: 0.418079
Elapsed time 165.5453450679779 seconds

EPOCH 3
Running average loss: 0.316517
VALIDATION LOSS: 0.414079
Elapsed time 164.26730871200562 seconds

EPOCH 4
Running average loss: 0.312620
VALIDATION LOSS: 0.412958
Elapsed time 164.86634159088135 seconds

EPOCH 5
Running average loss: 0.312303
VALIDATION LOSS: 0.412623
Elapsed time 163.28610181808472 seconds

EPOCH 6
Running average loss: 0.311784
VALIDATION LOSS: 0.412160
Elapsed time 162.84262919425964 seconds

EPOCH 7
Running average loss: 0.310460
VALIDATION LOSS: 0.411863
Elapsed time 163.71142554283142 seconds

EPOCH 8
Running average loss: 0.312920
VALIDATION LOSS: 0.410998
Elapsed time 164.06922149658203 seconds

EPOCH 9
Running average loss: 0.317150
VALIDATION LOSS: 0.411153
Elapsed time 163.56079769134521 seconds

EPOCH 10
Elapsed time 60.60630416870117 second

KeyboardInterrupt: 

In [32]:
#save_best_path = base_dir+'rnn_v2_predictor_lr2_weights.mdl'
#loss_func = nn.L1Loss()
#model = rnn.RNN(loss_func=loss_func).cuda()
#model.load_state_dict(torch.load(save_best_path))

In [17]:
outputs, targets, sequences = model.predict(testloader)

In [54]:
fp = '../data/submissions/rnn_v2.csv'
submissions.write_submission(outputs, fp)